<a href="https://colab.research.google.com/github/arnaldocavalcanti/14ia_GCP/blob/main/ML_com_GCP_02_Criar_Dataset_Atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autenticação
Primeiro precisamos autenticar nossa sessão do Colab no Google e definimos o ID do projeto

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
#@title Informe o id do projeto nesta linha e o bucket:

project_id = "arnaldo-creditosgcp" #@param {type:"string"}
bucket_id = "arnaldo-creditosgcp-14ia" #@param {type:"string"}


# Client Big Query

Como vamos buscar dados no bigquery, é necessário criar um cliente BG

In [3]:
# Call BigQuery and examine in dataframe
from google.cloud import bigquery

bgclient = bigquery.Client(project=project_id)

Demais imports:

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns, numpy as np


#Obtenção dos dados

Como vimos, temos 137.826.763 de observações na base.

Mesmo considerando apenas os nascimentos a partir de 2001, teremos 33.271.914 observações.

Vamos faça um comando para obter apenas 0,1% porcento do registros (ou seja, **aproximadamente** 33.3 mil observações)

In [8]:
query = """
SELECT
  COUNT(1) as observacoes
FROM
  publicdata.samples.natality
WHERE year > 2001 
AND RAND() < 0.001
"""
df = bgclient.query(query).to_dataframe()
df

,observacoes
0,29105


In [ ]:
query = """
SELECT
  weight_pounds * 0.45359237 AS weight_kilos,
  is_male,
  mother_age,
  mother_married,
  CASE
      WHEN plurality = 1 THEN "1"
      WHEN plurality = 2 THEN "2"
      WHEN plurality = 3 THEN "3"
      WHEN plurality = 4 THEN "4"
      WHEN plurality = 5 THEN "5"
      WHEN plurality >= 6 THEN "6"
      ELSE NULL
  END AS plurality,
  gestation_weeks,
  CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)) as year_and_month,
  FARM_FINGERPRINT('ELTHON') AS MeuHash,  
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth,
  RAND() AS meurand
FROM
  publicdata.samples.natality
WHERE year > 2000 
AND RAND() < 0.001
"""

In [ ]:
df = bgclient.query(query).to_dataframe()
df.shape

(33364, 10)

In [ ]:
df.head()

# Pré-Processamento

Aparentemente, há observações com NaN ou ainda como valores igual a zero. Ajuste o dataset para que isso não "polua" nossa análise

In [ ]:
df.isnull().sum()

weight_kilos        36
is_male              0
mother_age           0
mother_married       0
plurality            0
gestation_weeks    254
year_and_month       0
MeuHash              0
hashmonth            0
meurand              0
dtype: int64

In [ ]:
df = df.dropna(how='any',axis=0)

In [ ]:
df[['is_male', 'mother_age', 'mother_married', 'plurality', 'gestation_weeks']].head()

,is_male,mother_age,mother_married,plurality,gestation_weeks
0,True,28,False,1,39.0
1,False,39,True,1,37.0
2,True,25,False,1,37.0
3,True,27,True,1,36.0
4,True,33,True,1,38.0


# Spliting em treino e teste

Divida o dataset em 75% treino e 25% testes.
Mantenha as colunas 

```
'weight_kilos', 'is_male', 'mother_age', 'mother_married', 'plurality', 'gestation_weeks', 'hashmonth'
```

In [ ]:
from sklearn.model_selection import train_test_split
df_treino, df_teste = train_test_split(df, test_size=0.25)

In [ ]:
print(df_treino.shape)
df_teste.shape

(24811, 10)


(8271, 10)

In [ ]:
df_treino = df_treino.loc[: , ['weight_kilos', 'is_male', 'mother_age', 'mother_married', 'plurality', 'gestation_weeks', 'hashmonth']]
df_teste = df_teste.loc[: , ['weight_kilos', 'is_male', 'mother_age', 'mother_married', 'plurality', 'gestation_weeks', 'hashmonth']]

In [ ]:
print(df_treino.shape, df_teste.shape)

(24811, 7) (8271, 7)


#Export dos dados para serem processados pelo Tensorflow


Grave o conteúdo em dois datasets, treino.csv e teste.csv, sem índice de linhas ou header de colunas

In [ ]:
df_treino.to_csv('treino.csv', index=False, header=False)
df_teste.to_csv('teste.csv', index=False, header=False)

Verifica os arquicos salvos:

In [ ]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   8271 teste.csv
  24811 treino.csv
  33082 total
==> teste.csv <==
3.4589999970993155,False,35,True,1,40.0,2599641534368510197
1.7009999985735575,True,18,False,1,34.0,7773938200482214258
2.8439999976150485,False,21,False,1,25.0,7420272703711713305
3.3449999971949147,True,35,True,1,37.0,260598435387740869
3.1379999973685027,True,18,False,1,35.0,6749419160304400104
3.0049999974800357,False,28,False,1,42.0,524531196325542205
2.6929999977416754,True,23,True,1,36.0,270792696282171059
3.543999997028035,True,31,True,1,41.0,1195438672706281328
3.304999997228458,False,33,True,1,39.0,7361509899869211058
2.7929999976578164,True,27,True,1,38.0,124458947937444850

==> treino.csv <==
2.3249999980502767,True,26,True,1,37.0,8387817883864991792
3.599999996981074,True,32,True,1,39.0,1622638268154624360
1.2159999989802739,True,16,False,1,30.0,4331749811015213748
2.799999997651946,False,23,False,1,40.0,1891060869255459203
2.862999997599115,False,21,False,1,40.0,1569531340167098963
3.7709999968376744,Fal

# Exporta para o Cloud Storage

In [ ]:
!gsutil cp *.csv gs://$bucket_id/fiap-plat/dados_treino_local

Copying file://teste.csv [Content-Type=text/csv]...
Copying file://treino.csv [Content-Type=text/csv]...
/ [2 files][  1.9 MiB/  1.9 MiB]                                                
Operation completed over 2 objects/1.9 MiB.                                      
